# Lab 1.3: Exploratory Data Analysis (EDA) with Pandas

## Outline

* Introduction to Pandas
* EDA using Pandas

Exploratory data analysis is a first crucial step to building predictive models from your data. EDA allows you
to confirm or invalidate some of the assumptions you are making about your data and understand relationships between your variables.

## Getting Started with [Pandas](http://pandas.pydata.org/pandas-docs/version/0.17.1/)

### What is Pandas?
A Python library providing data structures and data analysis tools.

### Huh?
Think of it like Excel for Python but better.

In [ ]:
#By convention import pandas like:
import pandas as pd

In [ ]:
#For fake data.
from numpy.random import randn
import numpy as np

## Series
Think of a Pandas Series as a _labeled_ one-dimensional vector. In fact, it need not be a numeric vector, it can contain arbitrary python objects.

In [ ]:
int_series = pd.Series(range(10))
int_series.head()

In [ ]:
num_series = pd.Series(randn(10))
num_series.head()

In [ ]:
str_series = pd.Series([x for x in 'abcde'*2])
str_series.head()

## Indexes
Notice how each series has an index (in this case a relatively meaningless default index).

Pandas can make great use of informative indexes. Indexes work similarly to a dictionary key, allowing fast lookups of the data associated with the index.

Indexes can also be exploited for fast group-bys, merges, time-series operations and lots more.

When you're really in the zone with pandas, you'll be thinking a lot about indexes.

In [ ]:
indexed_series = pd.Series(randn(5), 
                           index = ['California', 'Alabama', 
                                    'Indiana', 'Montana', 
                                    'Kentucky'])
alt_indexed_series = pd.Series(randn(5),
                               index = ['Washington', 'Alabama', 
                                        'Montana', 'Indiana', 
                                        'New York'])
print (indexed_series)
print ('\n')
print (alt_indexed_series)

In [ ]:
#Pandas uses the index by default to align series for arithmetic!
indexed_series + alt_indexed_series

In [ ]:
indexed_series = pd.Series(randn(5*10), 
                           index = ['California', 'Alabama', 
                                    'Indiana', 'Montana', 
                                    'Kentucky']*10)
#If you have non-unique indexes, you can use them 
#to do groupby operations.
indexed_series.groupby(level=0).mean()

In [ ]:
#Datetime index
dt_index = pd.date_range('2015-1-1', 
                        '2015-11-1', 
                        freq='m')
dt_series = pd.Series(randn(10), 
                      index = dt_index)
dt_series

In [ ]:
#Datetime indexes make it easy to transform freqs etc!
dt_series.resample('q').mean()

## DataFrames
Data frames extend the concept of Series to table-like data.

In [ ]:
df = pd.DataFrame(randn(10, 5), index=dt_index, columns=[x for x in 'abcde'])
df

In [ ]:
#A dataframes columns are series:
col = df.a
type(col)

In [ ]:
#So are the rows.
row = df.ix['2015-01-31']
type(row)

In [ ]:
#The columns all have the same index:
col.index   

In [ ]:
#What's the index for the rows?
row.index

## DataFrame basics

In [ ]:
#New column
df['new'] = df['a'] + df['b']
df

In [ ]:
#Delete a column
df.drop('new', axis=1)

## Axis?
Because pandas thinks of rows and columns as both being series, anything we can do to rows we can do to columns too. 

Axis describes which one we want to do it to. 0=rows, 1=columns.

In [ ]:
lbl = pd.Timestamp('2015-04-30 00:00:00', offset='M')
df.drop(lbl, axis=0)

## Selecting subsets
There a couple of ways to select a subset of a python data frame.

In [ ]:
#To subset the rows, you can use the convenient:
df[df.a > 0]

In [ ]:
#Or combine multiple conditions:
df[(df.a > 0) & (df.c < 0)]

In [ ]:
#Selecting a column
df.a
#Works sometimes.

In [ ]:
df['a'] #works always

In [ ]:
df[['a', 'b']]
#Or a subset of columns

## Advanced selection
The above methods work for simple types of selections, but what if you need something more complex?

In [ ]:
df.loc['2015-05-31':'2015-08-31', 'c':'e'] #Ranges by label.

In [ ]:
df.iloc[2:-3,2:5] #Ranges by number.

In [ ]:
# SQL-like queries for parsimonious row selection.
# Experimental
df.query('a > 1 and c <1')

In [ ]:
# Multi Index:
dt_index = pd.date_range('2015-1-1', 
                        '2017-7-1', 
                        freq='m')
df = pd.DataFrame(randn(30,5), index=dt_index)

In [ ]:
df['state'] = ['Alabama', 'Alaska' , 'Arizona'] * 10
df.head()

In [ ]:
df = df.reset_index()
df = df.set_index(['state', 'index'])
df.head()

In [ ]:
df.loc['Alabama'].head()

In [ ]:
df.loc['2015-01-31']  # Doesn't work.

In [ ]:
df.loc[('Alabama', '2015-01-31')]  # Can do this.

In [ ]:
# Can also have multi-index in columns.
df.reset_index().set_index(['index', 'state']).unstack().head()

In [ ]:
df.head()

## split-apply-combine

In [ ]:
df.groupby(level=1).mean().head()

In [ ]:
df.groupby(level=0).mean().head()  # Groupby index

In [ ]:
# Groupby doesn't have to be on index.
df.reset_index().groupby('state').mean().head()
# 'state' could be a list of columns to group on.

In [ ]:
# Apply
g = df.groupby(level=0)

In [ ]:
# If one row for each index.
g.aggregate(np.mean)

In [ ]:
# If same shape as original:
g.transform(lambda x: (x - x.mean())/x.std()).head()

## Reading and writing data

In [ ]:
df.to_csv('saved_data.csv', delimiter='\t')

In [ ]:
read = pd.read_csv('saved_data.csv', delimiter='\t')
#TONS OF options for reading data

"""Other methods:
pd.read_excel
pd.read_sql
pd.read_stata
...
"""

## Other very useful things

join, concat and merge

In [ ]:
state_df = pd.DataFrame({'governor':['Robert Bentley',
                                    'Bill Walker',
                                    'Doug Ducey',
                                    'Asa Hutchinson']}, 
                        index=['Alabama', 'Alaska', 'Arizona', 'Arkansas'])
#Note merge is most useful when you want to merge on something other than the index.
#Default is to merge on common column names.
pd.merge(df.reset_index(), state_df, 
         left_on='state', right_index=True, how='right').tail()

In [ ]:
state_avg = df.groupby(level=0).mean()
state_avg.head()

In [ ]:
#Concat allows joining along the axes.
pd.concat([state_avg, state_df], axis=1)

In [ ]:
#Join also works. Just like merge, but 
#default is to join on indexes.
state_avg.join(state_df, how='right')

## Exploratory Data Analysis with Pandas

In [ ]:
%pylab inline

In [ ]:
df = pd.read_csv('https://s3-us-west-2.amazonaws.com/dsci/6002/data/playgolf.csv', delimiter='|' )
df.head()

## Describe the continuous variables
### This treats the Boolean Windy variable as a series of 0's and 1's

In [ ]:
df.describe()

Can see the general pattern of Temperature and Humidity and mean of a Boolean represents the percentage

### We can make use of df.plot() to produce simple graphs that calls on the more adjustable [Matplotlib](http://matplotlib.org/api/pyplot_api.html) library 

In [ ]:
df.hist(['Temperature','Humidity'],bins=5)

In [ ]:
df[['Temperature','Humidity']].plot(kind='box')

### Scatterplots for examining bivariate relationships

In [ ]:
df.plot('Temperature', 'Humidity', kind='scatter');

## What about the categorical variables? Frequency tables and relative frequency tables

### Simply df.value_counts() gets you the frequencies

In [ ]:
df['Outlook'].value_counts()

### Using apply will get you the value counts for multiple columns at once

In [ ]:
df[['Outlook','Result']].apply(lambda x: x.value_counts())

### Contingency Tables for looking at bivariate relationships between two categorical variables

In [ ]:
pd.crosstab(df['Outlook'], df['Result'])

### Often we want the row percentages

In [ ]:
pd.crosstab(df['Outlook'], df['Result']).apply(lambda r: r/r.sum(), axis=1)

### Or the column percentages

In [ ]:
pd.crosstab(df['Outlook'], df['Result']).apply(lambda c: c/c.sum(), axis=0)

## Lab Exercises [GRADED]

In this scenario, you are a data scientist at [Bay Area Bike Share](http://www.bayareabikeshare.com/). Your task
is to provide insights on bike user activity and behavior to the products team. 


1. Load the `trips` table into a dataframe. 
   
   Make 4 extra columns from the `start_date` column (We will use these in later questions):
   - `month` would contain only the month component
   - `dayofweek` would indicate what day of the week the date is
   - `date` would contain only the date component 
   - `hour` would only contain the hour component
   - [Hint to deal with datetime objects in pandas](http://stackoverflow.com/questions/25129144/pandas-return-hour-from-datetime-column-directly)

2. Group the bike rides by `month` and count the number of users per month. Plot the number of users for each month. 
   What do you observe? Provide a likely explanation to your observation. Real life data can often be messy/incomplete
   and cursory EDA is often able to reveal that.
   
3. Plot the daily user count from September to December. Mark the `mean` and `mean +/- 1.5 * Standard Deviation` as 
   horizontal lines on the plot. This would help you identify the outliers in your data. Describe your observations. 
   
   ![image](images/timeseries.png)

4. Plot the distribution of the daily user counts for all months as a histogram. Comment on the distribution you see and explain why the distribution might be shaped as such. 
    
   <img src="images/hist.png" width="500">
  
5. Now we are going to explore hourly trends of user activity. Group the bike rides by `date` and `hour` and count 
   the number of rides in the given hour on the given date. Make a 
   [boxplot](http://blog.bharatbhole.com/creating-boxplots-with-matplotlib/) of the hours in the day **(x)** against
   the number of users **(y)** in that given hour. 
   
6. Someone from the analytics team made a line plot (_right_) that he claims is showing the same information as your
   boxplot (_left_). What information can you gain from the boxplot that is missing in the line plot?
   
   ![image](images/q1_pair.png)

7. ** Extra Credit **: Replot the boxplot in `6.` after binning your data into weekday and weekend. Describe the differences you observe between hour user activity between weekday and weekend? 
    

# Research for tomorrow:
## What is Anscombe's Quartet?
### Be prepared to explain it tomorrow at the beginning of class.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
%pylab inline  
import yaml

Populating the interactive namespace from numpy and matplotlib


In [2]:
%load_ext sql

//anaconda/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
//anaconda/lib/python3.5/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [3]:
pg_creds = yaml.load(open('../../pg_creds.yaml'))['student']
connection_string = 'postgresql://{user}:{password}@{host}:{port}/{dbname}'.format(**pg_creds)

In [4]:
# Creates a connection to our Postgres database
%sql $connection_string

'Connected: student@postgres'

In [5]:
# loads the data from the trips table into a Pandas DataFrame
trips_table = %sql SELECT * FROM trips;
trips = trips_table.DataFrame()

144015 rows affected.


In [6]:
trips.head()

,trip_id,duration,start_date,start_station,start_terminal,end_date,end_station,end_terminal,bike_#,subscription_type,zip_code
0,4576,63,2013-08-29 14:13:00,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127
1,4607,70,2013-08-29 14:42:00,San Jose City Hall,10,8/29/2013 14:43,San Jose City Hall,10,661,Subscriber,95138
2,4130,71,2013-08-29 10:16:00,Mountain View City Hall,27,8/29/2013 10:17,Mountain View City Hall,27,48,Subscriber,97214
3,4251,77,2013-08-29 11:29:00,San Jose City Hall,10,8/29/2013 11:30,San Jose City Hall,10,26,Subscriber,95060
4,4299,83,2013-08-29 12:02:00,South Van Ness at Market,66,8/29/2013 12:04,Market at 10th,67,319,Subscriber,94103


In [7]:
trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144015 entries, 0 to 144014
Data columns (total 11 columns):
trip_id              144015 non-null int64
duration             144015 non-null int64
start_date           144015 non-null datetime64[ns]
start_station        144015 non-null object
start_terminal       144015 non-null int64
end_date             144015 non-null object
end_station          144015 non-null object
end_terminal         144015 non-null int64
bike_#               144015 non-null int64
subscription_type    144015 non-null object
zip_code             137885 non-null object
dtypes: datetime64[ns](1), int64(5), object(5)
memory usage: 12.1+ MB
